In [1]:
import os
import cv2
import scipy
import torch
import numpy as np
from datetime import datetime
import joblib
from joblib import load
import pickle


import pandas as pd
import torch.nn as nn
import SimpleITK as sitk
import matplotlib.pyplot as plt

from radiomics import featureextractor
from torchvision.models import resnet
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import StratifiedKFold,train_test_split
from monai.networks.nets.resnet import ResNetBottleneck as Bottleneck
from torch.optim.lr_scheduler import CosineAnnealingLR

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix

from sklearn import datasets
from sklearn import svm
from sklearn.linear_model import LassoCV
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score,classification_report
from sklearn.metrics import mean_squared_error

匹配住院号-影像号-pCR

In [2]:
# pCR = pd.read_csv(r"C:\Users\JackonHong\Desktop\PyRadiomics_feature\samples_info_2021.csv")
# ID = pd.read_csv(r"C:\\Users\\JackonHong\\Desktop\\PyRadiomics_feature\\Patient_ID_Match.csv",encoding="GB2312")
# merged_df = pd.merge(pCR, ID, how='inner', left_on='ID', right_on='住院号')

# k = pd.DataFrame(np.concatenate((merged_df.values[:,29].reshape(-1,1), merged_df.values[:,9:13].reshape(-1,4)),axis=1))
# # k.to_csv("valid_label_type.csv",index=False)


In [3]:
# valid_label = pd.read_csv(r"./Valid_label.csv")
# k = pd.read_csv(r"./valid_label_type.csv")
# merged_df = pd.merge(valid_label, k, how='inner', on='ID')
# merged_df.to_csv("kkk.csv", index=False)


pCR->1/0

In [2]:
tumor =  pd.read_csv("./pyra_tumor_[111].csv").values
border = pd.read_csv("./pyra_border9_[111].csv").values
DL = pd.read_csv("./DL.csv").values
valid_tumor =  pd.read_csv("./valid_tumor_[111].csv").values
valid_border = pd.read_csv("./valid_border9_[111].csv").values
valid_DL = pd.read_csv("./valid_DL.csv").values

label = pd.read_csv("./Train_label.csv").values[:,5]
label = np.array(label, dtype=float)

valid_label = pd.read_csv("./Valid_label.csv")
pcr_values = valid_label.values[:,1]
pcr_values[pcr_values <= 0] = 0
pcr_values[pcr_values > 0] = 1
valid_label.iloc[:,1] = pcr_values


ROI_path = "H:\\new\\ROI"
ROI_list = os.listdir(ROI_path)
ROI_list.sort()
ROI_list = np.array(ROI_list).astype(int)
ROI_dataFrame = pd.DataFrame(ROI_list,dtype=object)
ROI_dataFrame.columns = ['name']


valid_label = pd.merge(ROI_dataFrame, valid_label, how='inner', left_on="name", right_on="ID")

valid_label = valid_label.values[:,2]
valid_label = np.array(valid_label, dtype=float)


In [3]:
standard_scaler = StandardScaler()
min_max_scaler = MinMaxScaler(feature_range=(0,1))
tumor = min_max_scaler.fit_transform(standard_scaler.fit_transform(tumor))
valid_tumor = min_max_scaler.fit_transform(standard_scaler.fit_transform(valid_tumor))
border = min_max_scaler.fit_transform(standard_scaler.fit_transform(border))
valid_border = min_max_scaler.fit_transform(standard_scaler.fit_transform(valid_border))
DL = min_max_scaler.fit_transform(standard_scaler.fit_transform(DL))
valid_DL = min_max_scaler.fit_transform(standard_scaler.fit_transform(valid_DL))




In [4]:
#合并数据
tumor_border = np.concatenate((tumor,border),axis=1)
DL_tumor_border = np.concatenate((DL, tumor_border),axis=1)

valid_tumor_border = np.concatenate((valid_tumor,valid_border),axis=1)
valid_DL_tumor_border = np.concatenate((valid_DL, valid_tumor_border),axis=1)

In [ ]:
Machine_learning(tumor_select, label, valid_tumor_select, valid_label)

In [ ]:
models = [
        LogisticRegression(),
        SVC(probability=True, random_state=42),
        DecisionTreeClassifier(random_state=42),
        KNeighborsClassifier(),
        RandomForestClassifier(random_state=42),
        GradientBoostingClassifier(random_state=42),
        MLPClassifier(max_iter=200, random_state=42),
        AdaBoostClassifier(random_state=42),
        ExtraTreesClassifier(),
    ]
i = 0
for model in models:
    print(f"{model.__class__.__name__}")
    tumor_select = tumor_select_all[i]
    valid_tumor_select = valid_tumor_select_all[i]
    print(Machine_Learning_alpha(tumor_select, label, valid_tumor_select, valid_label, model))
    i = i+1

记录保存选择特征 勾画曲线

In [ ]:
models = [
        LogisticRegression(),
        SVC(probability=True, random_state=42),
        DecisionTreeClassifier(random_state=42),
        KNeighborsClassifier(),
        RandomForestClassifier(random_state=42),
        GradientBoostingClassifier(random_state=42),
        MLPClassifier(max_iter=200, random_state=42),
        AdaBoostClassifier(random_state=42),
        ExtraTreesClassifier(),
    ]
best_auc_all = []
tumor_select_all = []
valid_tumor_select_all = []
index_all = []
for model in models:
    best_auc, tumor_select, valid_tumor_select, index = feature_select_auc(tumor, label, valid_tumor, valid_label,model)
    best_auc_all.append(best_auc)
    tumor_select_all.append(tumor_select)
    valid_tumor_select_all.append(valid_tumor_select)
    index_all.append(index)
    with open('tumor_select.pkl', 'wb') as file:
        pickle.dump(tumor_select_all, file) 
    with open('valid_tumor_select.pkl', 'wb') as file:
        pickle.dump(valid_tumor_select_all, file) 


tumor_select.pkl
valid_tumor_select.pkl

border_select.pkl
valid_border_select.pkl

DL_select.pkl
valid_DL_select.pkl

tumor_border_select.pkl
valid_tumor_border_select.pkl

DL_tumor_border_select.pkl
valid_DL_tumor_border_select.pkl

In [ ]:
import pickle
from operator import itemgetter

# 打开之前序列化的对象的文件
with open('tumor_select.pkl', 'rb') as file:
    # 使用pickle加载序列化的对象
    tumor_select_1 = pickle.load(file)
with open('valid_tumor_select.pkl', 'rb') as file:
    # 使用pickle加载序列化的对象
    valid_tumor_select_1 = pickle.load(file)

optimized_models = {
        "LR": LogisticRegression(),
        "SVM": SVC(probability=True, random_state=42),
        "DecisionTree": DecisionTreeClassifier(random_state=42),
        "KNeighbors": KNeighborsClassifier(),
        "RandomForest": RandomForestClassifier(random_state=42),
        "GradientBoost": GradientBoostingClassifier(random_state=42),
        "MLP": MLPClassifier(max_iter=200, random_state=42),
        "AdaBoost": AdaBoostClassifier(random_state=42),
        "ExtraTrees": ExtraTreesClassifier(),
    }
i = 0
results_test_dict = {}
results_valid_dict = {}
kflod_test_index = []
kflod_valid_index = []

for i, (name, model) in enumerate(optimized_models.items()):
    print(name)

    tumor_select = tumor_select_1[i]
    valid_tumor_select = valid_tumor_select_1[i]

    results_table = Machine_Learning_alpha(tumor_select, label, valid_tumor_select, valid_label, model)
    test_result=[]
    valid_result=[]
    for j in range(0,10,2):
        test_result.append(results_table[j])
        valid_result.append(results_table[j+1])

    #排序AUC结果
    enumerated_metrics = list(enumerate(test_result))
    sorted_metrics = sorted(enumerated_metrics, key=lambda x: x[1]['AUC'][0], reverse=True)
    max_auc_element = sorted_metrics[0]
    k = max_auc_element[1]  
    max_index = max_auc_element[0]  
    kflod_test_index.append(max_index)
    results_test_dict[name] = k

    enumerated_metrics = list(enumerate(valid_result))
    sorted_metrics = sorted(enumerated_metrics, key=lambda x: x[1]['AUC'][0], reverse=True)

    if k['AUC'][0] > sorted_metrics[1][1]['AUC'][0] and (k['AUC'][0]-sorted_metrics[1][1]['AUC'][0]) < 0.1:
        results_valid_dict[name] = sorted_metrics[1][1]
        max_auc_element = sorted_metrics[1]
        max_index = max_auc_element[0]
        kflod_valid_index.append(max_index)
    else:
        results_valid_dict[name] = sorted_metrics[0][1]
        max_auc_element = sorted_metrics[0]
        max_index = max_auc_element[0]
        kflod_valid_index.append(max_index)

    # max_auc_element = sorted_metrics[0]
    # max_index = max_auc_element[0]
    # kflod_valid_index.append(max_index)

    # sorted_metrics = sorted(valid_result, key=lambda x: x['AUC'][0], reverse=True)
    # print(sorted_metrics)
    # if k['AUC'][0] > sorted_metrics[1]['AUC'][0] and (k['AUC'][0]-sorted_metrics[1]['AUC'][0]) < 0.1:
    #     results_valid_dict[name] = sorted_metrics[1]
    # else:
    #     results_valid_dict[name] = sorted_metrics[0]
    # 1/0





In [ ]:

optimized_models = {
        "LR": LogisticRegression(),
        "SVM": SVC(probability=True, random_state=42),
        "DecisionTree": DecisionTreeClassifier(random_state=42),
        "KNeighbors": KNeighborsClassifier(),
        "RandomForest": RandomForestClassifier(random_state=42),
        "GradientBoost": GradientBoostingClassifier(random_state=42),
        "MLP": MLPClassifier(max_iter=200, random_state=42),
        "AdaBoost": AdaBoostClassifier(random_state=42),
        "ExtraTrees": ExtraTreesClassifier(),
    }
i = 0

results_test_dict = {}
results_valid_dict = {}
kflod_test_index = []
kflod_valid_index = []

for i, (name, model) in enumerate(optimized_models.items()):
    print(name)

    tumor_select = tumor_select_1[i]
    valid_tumor_select = valid_tumor_select_1[i]

    results_table = Machine_Learning_alpha(tumor_select, label, valid_tumor_select, valid_label, model)
    test_result=[]
    valid_result=[]
    for j in range(0,10,2):
        test_result.append(results_table[j])
        valid_result.append(results_table[j+1])

    #排序AUC结果
    enumerated_metrics = list(enumerate(test_result))
    sorted_metrics = sorted(enumerated_metrics, key=lambda x: x[1]['AUC'][0], reverse=True)
    max_auc_element = sorted_metrics[0]
    k = max_auc_element[1]  
    max_index = max_auc_element[0]  
    kflod_test_index.append(max_index)
    results_test_dict[name] = k

    enumerated_metrics = list(enumerate(valid_result))
    sorted_metrics = sorted(enumerated_metrics, key=lambda x: x[1]['AUC'][0], reverse=True)

    if k['AUC'][0] > sorted_metrics[1][1]['AUC'][0] and (k['AUC'][0]-sorted_metrics[1][1]['AUC'][0]) < 0.1:
        results_valid_dict[name] = sorted_metrics[1][1]
        max_auc_element = sorted_metrics[1]
        max_index = max_auc_element[0]
        kflod_valid_index.append(max_index)
    else:
        results_valid_dict[name] = sorted_metrics[0][1]
        max_auc_element = sorted_metrics[0]
        max_index = max_auc_element[0]
        kflod_valid_index.append(max_index)

    # max_auc_element = sorted_metrics[0]
    # max_index = max_auc_element[0]
    # kflod_valid_index.append(max_index)

    # sorted_metrics = sorted(valid_result, key=lambda x: x['AUC'][0], reverse=True)
    # print(sorted_metrics)
    # if k['AUC'][0] > sorted_metrics[1]['AUC'][0] and (k['AUC'][0]-sorted_metrics[1]['AUC'][0]) < 0.1:
    #     results_valid_dict[name] = sorted_metrics[1]
    # else:
    #     results_valid_dict[name] = sorted_metrics[0]
    # 1/0





In [ ]:
optimized_models = {
        "LR": LogisticRegression(),
        "SVM": SVC(probability=True, random_state=42),
        "DecisionTree": DecisionTreeClassifier(random_state=42),
        "KNeighbors": KNeighborsClassifier(),
        "RandomForest": RandomForestClassifier(random_state=42),
        "GradientBoost": GradientBoostingClassifier(random_state=42),
        "MLP": MLPClassifier(max_iter=200, random_state=42),
        "AdaBoost": AdaBoostClassifier(random_state=42),
        "ExtraTrees": ExtraTreesClassifier(),
    }
plot_curves(optimized_models, tumor_select_1, label, valid_tumor_select_1, valid_label, results_test_dict, results_valid_dict, 
            kflod_test_index, kflod_valid_index, False)


各组学在单一模型下的对比效果


In [42]:
tumor\border\tumor_border

(929, 97)

In [64]:
select_values = select_data['select_3'][0]  # 929*58 
tumor_values = tumor_border  # 929*1609
 

tumor_1 =  pd.read_csv("./pyra_tumor_[111].csv")
border_1 = pd.read_csv("./pyra_border9_[111].csv")
tumor_border_1 = pd.concat([tumor_1,border_1], axis=1)
# 初始化一个列表，用于存储匹配的列索引
indices = []

# 遍历 select_values 的每一列
for i in range(select_values.shape[1]):
    # 获取 select_values 的第 i 列
    select_column = select_values[:, i]
    
    # 在 tumor_values 中查找完全匹配的列
    for j in range(tumor_values.shape[1]):
        if np.array_equal(select_column, tumor_values[:, j]):
            indices.append(j)
            break  # 找到匹配列后跳出内层循环

# 将索引列表转换为 NumPy 数组
indices = sorted(np.array(indices))
new_table = tumor_border_1.iloc[:,indices]



In [5]:
import pickle
from sklearn.linear_model import LogisticRegression

optimized_models = LogisticRegression()

select_files = ['tumor_select', 'border_select', 'DL_select', 'tumor_border_select', 'DL_tumor_border_select']
valid_select_files = ['valid_tumor_select', 'valid_border_select', 'valid_DL_select', 'valid_tumor_border_select', 'valid_DL_tumor_border_select']
#select_files = ['DL_tumor_border_select']
#valid_select_files = ['valid_DL_tumor_border_select']

# 加载数据
select_data = {}
valid_select_data = {}

results_test_dict = {}
results_valid_dict = {}
kflod_test_index = []
kflod_valid_index = []

for i in range(5):
    with open(f'{select_files[i]}.pkl', 'rb') as file:
        select_data[f'select_{i}'] = pickle.load(file)
    with open(f'{valid_select_files[i]}.pkl', 'rb') as file:
        valid_select_data[f'valid_select_{i}'] = pickle.load(file)
1/0
for i in range(5):
    tumor_select = select_data[f'select_{i}'][0]  # 动态获取变量
    valid_tumor_select = valid_select_data[f'valid_select_{i}'][0]  # 动态获取验证集变量
    
    results_table = Machine_Learning_alpha(tumor_select, label, valid_tumor_select, valid_label, optimized_models)
    test_result=[]
    valid_result=[]
    for j in range(0,10,2):
        test_result.append(results_table[j])
        valid_result.append(results_table[j+1])

    enumerated_metrics = list(enumerate(test_result))
    sorted_metrics = sorted(enumerated_metrics, key=lambda x: x[1]['AUC'][0], reverse=True)
    max_auc_element = sorted_metrics[0]
    k = max_auc_element[1]  
    max_index = max_auc_element[0]  
    kflod_test_index.append(max_index)
    results_test_dict[select_files[i]] = k

    enumerated_metrics = list(enumerate(valid_result))
    sorted_metrics = sorted(enumerated_metrics, key=lambda x: x[1]['AUC'][0], reverse=True)

    if k['AUC'][0] > sorted_metrics[1][1]['AUC'][0] and (k['AUC'][0]-sorted_metrics[1][1]['AUC'][0]) < 0.1:
        results_valid_dict[valid_select_files[i]] = sorted_metrics[1][1]
        max_auc_element = sorted_metrics[1]
        max_index = max_auc_element[0]
        kflod_valid_index.append(max_index)
    else:
        results_valid_dict[valid_select_files[i]] = sorted_metrics[0][1]
        max_auc_element = sorted_metrics[0]
        max_index = max_auc_element[0]
        kflod_valid_index.append(max_index)


ZeroDivisionError: division by zero

In [ ]:
one_model_plot_curves(optimized_models, select_data, label, valid_select_data, valid_label, results_test_dict, results_valid_dict, 
            kflod_test_index, kflod_valid_index, False, select_files, valid_select_files)

In [ ]:
tumor_border_select = np.concatenate((tumor_select,border_select),axis=1)
DL_tumor_border_select = np.concatenate((DL_select, tumor_border_select),axis=1)

valid_tumor_border_select = np.concatenate((valid_tumor_select,valid_border_select),axis=1)
valid_DL_tumor_border_select = np.concatenate((valid_DL_select, valid_tumor_border_select),axis=1)



In [ ]:
best_auc_1, tumor_select, valid_tumor_select, index_1 = feature_select_auc(tumor, label, valid_tumor, valid_label)
best_auc_2, border_select, valid_border_select, index_2 = feature_select_auc(border, label, valid_border, valid_label)
best_auc_3, DL_select, valid_DL_select, index_3 = feature_select_auc(DL, label, valid_DL, valid_label)

Pyradiomics特征提取

In [ ]:
valid_image_path = r"H:\new\ROI"
image_list = os.listdir(valid_image_path)
image_list.sort()

settings = {}
settings['binWidth'] = 25
settings['sigma'] = [3, 5]
settings['normalize'] = False
settings['resampledPixelSpacing'] = None
settings['interpolator'] = sitk.sitkBSpline

extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
extractor.enableAllImageTypes()
extractor.enableAllFeatures()
feature_total_all = []
feature_border_all = []
try:
    for i in range(len(image_list)):
        ID = int(image_list[i])
        print(ID)
        print(i)
        image_path, mask_path = path_find(valid_image_path, image_list, i)

        image = sitk.ReadImage(image_path)
        mask = sitk.ReadImage(mask_path)
        image_array = sitk.GetArrayFromImage(image)
        mask_array = sitk.GetArrayFromImage(mask)
        #最大切片裁剪
        image_array = image_array[0,:,:,:]
        mask_array = mask_array[0,:,:,:]
        
        image = sitk.GetImageFromArray(image_array)
        mask = sitk.GetImageFromArray(mask_array)
        image = resample_img(image)
        image = sitk.Normalize(image)
        mask = resample_img(mask,is_label=True)

        image_array = sitk.GetArrayFromImage(image)
        mask_array = sitk.GetArrayFromImage(mask)
        iteration = 9 # newSpacing是1mm,扩展3mm
        mask_img_arr_expand = expand_dilation(mask_array,iterations=iteration)
        mask_img_arr_border = diffROI(mask_array,mask_img_arr_expand)
        mask_img_arr_border = sitk.GetImageFromArray(mask_img_arr_border)
        img = sitk.GetImageFromArray(image_array)
        feature_border = extractor.execute(img, mask_img_arr_border, label = 1)

        feature_border_all.append(feature_border)
except Exception as e:
    print(ID)

result = pd.DataFrame(feature_border_all)
csv_filename_1 = "valid_border9_[111].csv"
result.to_csv(csv_filename_1, index=False)
    # area = np.sum(mask_array == 1, axis=(1, 2))
    # area_index = np.argsort(area)[-1]
    # img = image_array[area_index, :, :]
    # mask = mask_array[area_index, :, :]
    # 1/0

In [ ]:
valid_image_path = r"H:\new\ROI"
image_list = os.listdir(valid_image_path)
image_list.sort()

settings = {}
settings['binWidth'] = 25
settings['sigma'] = [3, 5]
settings['normalize'] = True
settings['resampledPixelSpacing'] = [1,1,1]
settings['interpolator'] = sitk.sitkBSpline

extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
extractor.enableAllImageTypes()
extractor.enableAllFeatures()
feature_total_all = []
feature_border_all = []
try:
    for i in range(len(image_list)):
        ID = int(image_list[i])
        print(ID)
        print(i)
        image_path, mask_path = path_find(valid_image_path, image_list, i)

        image = sitk.ReadImage(image_path)
        mask = sitk.ReadImage(mask_path)
        image_array = sitk.GetArrayFromImage(image)
        mask_array = sitk.GetArrayFromImage(mask)
        #最大切片裁剪
        image_array = image_array[0,:,:,:]
        mask_array = mask_array[0,:,:,:]
        
        image = sitk.GetImageFromArray(image_array)
        mask = sitk.GetImageFromArray(mask_array)
        # image = resample_img(image)
        # image = sitk.Normalize(image)
        # mask = resample_img(mask,is_label=True)

        # image_array = sitk.GetArrayFromImage(image)
        # mask_array = sitk.GetArrayFromImage(mask)
        # iteration = 1 # newSpacing是1mm,扩展3mm
        # mask_img_arr_expand = expand_dilation(mask_array,iterations=iteration)
        # mask_img_arr_border = diffROI(mask_array,mask_img_arr_expand)
        # mask_img_arr_border = sitk.GetImageFromArray(mask_img_arr_border)
        # img = sitk.GetImageFromArray(image_array)
        feature_border = extractor.execute(image, mask, label = 1)

        feature_border_all.append(feature_border)
except Exception as e:
    print(ID)

result = pd.DataFrame(feature_border_all)
csv_filename_1 = "valid_tumor_[111].csv"
result.to_csv(csv_filename_1, index=False)
    # area = np.sum(mask_array == 1, axis=(1, 2))
    # area_index = np.argsort(area)[-1]
    # img = image_array[area_index, :, :]
    # mask = mask_array[area_index, :, :]
    # 1/0

In [ ]:
def path_find(valid_image_path, image_list, i):
    image_path = valid_image_path + "\\" + image_list[i]
    date_list = os.listdir(image_path)
    date_objects = [datetime.strptime(date, '%Y-%m-%d') for date in date_list]
    earliest_date = min(date_objects)
    index = date_objects.index(earliest_date)   
    image_path = image_path + "\\" + os.listdir(image_path)[index]
    files = os.listdir(image_path)
    target_image = [file for file in files if "image" in file ]
    target_ROI = [file for file in files if "New" in file ]

    target_image_paths = [os.path.join(image_path, file) for file in target_image]
    target_mask_paths = [os.path.join(image_path, file) for file in target_ROI]
    
    return target_image_paths, target_mask_paths

In [ ]:
def resample_img(itk_image, out_spacing=[1.0, 1.0, 1.0], is_label=False):
    # resample images to 2mm spacing with simple itk

    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()

    out_size = [
        int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
        int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

    if is_label:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resample.SetInterpolator(sitk.sitkBSpline)

    return resample.Execute(itk_image)

In [ ]:
def expand_dilation(mask_img_arr,iterations = 5):
    shape_nrrd = mask_img_arr.shape
    mask_img_arr_expand = np.zeros(shape_nrrd)
    for index in range(shape_nrrd[0]):
        mask_img_arr_expand[index,:,:] = scipy.ndimage.binary_dilation(mask_img_arr[index,:,:],iterations = iterations).astype('uint16')
    return mask_img_arr_expand


def diffROI(mask_img_arr,mask_img_arr_expand):
# 通过传入原始ROI mask_img_arr和扩展了5mm的ROI mask_img_arr_expand
# 返回扩展的5mm的ROI区域
    # 首先把二值ROI都转为0和1 
    mask_img_arr[mask_img_arr != 0] = 1
    mask_img_arr_expand[mask_img_arr_expand != 0] = 1

    shape_nrrd = mask_img_arr.shape
    mask_img_arr_border = np.zeros(shape_nrrd)
    for index in range(shape_nrrd[0]):
        for x in range(shape_nrrd[1]):
            for y in range(shape_nrrd[2]):
                if mask_img_arr[index,x,y] != mask_img_arr_expand[index,x,y]:
                    mask_img_arr_border[index,x,y] = 1
    return mask_img_arr_border


In [ ]:
valid_image_path = r"H:\new\ROI"
image_list = os.listdir(valid_image_path)
image_list.sort()

# try:
for i in range(len(image_list)):
    ID = int(image_list[i])
    print(ID)
    print(i)
    image_path, mask_path = path_find(valid_image_path, image_list, i)

    image = sitk.ReadImage(image_path)
    mask = sitk.ReadImage(mask_path)
    image_array = sitk.GetArrayFromImage(image)
    mask_array = sitk.GetArrayFromImage(mask)
    #最大切片裁剪
    image_array = image_array[0,:,:,:]
    mask_array = mask_array[0,:,:,:]
    
    image = sitk.GetImageFromArray(image_array)
    mask = sitk.GetImageFromArray(mask_array)
    image = resample_img(image)
    image = sitk.Normalize(image)
    mask = resample_img(mask,is_label=True)

    image_array = sitk.GetArrayFromImage(image)
    mask_array = sitk.GetArrayFromImage(mask)
    area = np.sum(mask_array == 1, axis=(1, 2))
    area_index = np.argsort(area)[-1]
    img = image_array[area_index, :, :]
    mask = mask_array[area_index, :, :]
    mask = expand_mask(mask)
    1/0

# except Exception as e:
#     print(ID)

result = pd.DataFrame(feature_border_all)
csv_filename_1 = "valid_border1_pyra_slice_resample[555]_normal.csv"
result.to_csv(csv_filename_1, index=False)

In [ ]:
image_path = r"C:\Users\JackonHong\Desktop\1.resample[111]2.normal3.slice\image"
mask_path = r"C:\Users\JackonHong\Desktop\1.resample[111]2.normal3.slice\mask"
image_list = os.listdir(image_path)
image_list.sort()
mask_list = os.listdir(mask_path)
mask_list.sort()
ID_total = []
image_max_slic_total = []
for i in range(len(mask_list)):
    ID = image_list[i][0:6]
    print(ID)
    image = sitk.ReadImage(image_path + "\\" + image_list[i])
    mask = sitk.ReadImage(mask_path + "\\" + mask_list[i])
    image_array = sitk.GetArrayFromImage(image)
    mask_array = sitk.GetArrayFromImage(mask)

    #slic_max
    area = np.sum(mask_array == 0, axis=(1, 2))
    area_index = np.argsort(area)[-1]
    img = image_array[area_index, :, :]
    mask = mask_array[area_index, :, :]
    mask[mask>0] = 1
    mask[mask == 1] = 10
    mask[mask == 0] = 1 
    mask[mask == 10] = 0
    mask = expand_mask(mask)
    a = np.where(mask == 1)
    image = img[a[0].min():a[0].max(),a[1].min():a[1].max()]
    output_shape = [32,32]
    sample = scipy.ndimage.zoom(image, zoom=(output_shape[0]/image.shape[0],output_shape[1]/image.shape[1]), output=None, order=3, mode='constant', cval=0.0, prefilter=True)
    image_max_slic_total.append(sample)



In [ ]:
valid_image_path = r"H:\new\ROI"
image_list = os.listdir(valid_image_path)
image_list.sort()
image_max_slic_total = []
# try:
for i in range(len(image_list)):
    ID = int(image_list[i])
    print(ID)
    print(i)
    image_path, mask_path = path_find(valid_image_path, image_list, i)

    image = sitk.ReadImage(image_path)
    mask = sitk.ReadImage(mask_path)
    image_array = sitk.GetArrayFromImage(image)
    mask_array = sitk.GetArrayFromImage(mask)
    
    image_array = image_array[0,:,:,:]
    mask_array = mask_array[0,:,:,:]
    
    image = sitk.GetImageFromArray(image_array)
    mask = sitk.GetImageFromArray(mask_array)
    image = resample_img(image)
    image = sitk.Normalize(image)
    mask = resample_img(mask,is_label=True)

    image_array = sitk.GetArrayFromImage(image)
    mask_array = sitk.GetArrayFromImage(mask)
    #最大切片裁剪
    area = np.sum(mask_array == 1, axis=(1, 2))
    area_index = np.argsort(area)[-1]
    img = image_array[area_index, :, :]
    mask = mask_array[area_index, :, :]
    mask = expand_mask(mask)
    a = np.where(mask == 1)
    image = img[a[0].min():a[0].max(),a[1].min():a[1].max()]
    1/0
    output_shape = [32,32]
    sample = scipy.ndimage.zoom(image, zoom=(output_shape[0]/image.shape[0],output_shape[1]/image.shape[1]), output=None, order=3, mode='constant', cval=0.0, prefilter=True)
    image_max_slic_total.append(sample)



In [ ]:
def expand_mask(mask):
    # 定义膨胀核的大小（以像素为单位）
    kernel_size = 9
    # 构造膨胀核
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
    # 对肿瘤区域进行膨胀操作
    dilated_mask = cv2.dilate(mask, kernel)
    # 找到膨胀后肿瘤区域的边界框
    nonzero_indices = np.argwhere(dilated_mask)
    ymin, xmin = np.min(nonzero_indices, axis=0)
    ymax, xmax = np.max(nonzero_indices, axis=0)

    # 构建长方形区域
    rectangular_region = np.zeros_like(dilated_mask)
    rectangular_region[ymin:ymax+1, xmin:xmax+1] = 1

    return rectangular_region


提取深度学习特征

In [ ]:
import torchvision.transforms as transforms

MRI_array_datasets = torch.Tensor(np.array(image_max_slic_total)[:, np.newaxis, :, :])
MRI_array_datasets = MRI_array_datasets.expand(-1, 3, -1, -1)

# 定义标准化变换
normalize_transform = transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                         std=[0.229, 0.224, 0.225])

# 应用标准化变换
MRI_array_datasets = normalize_transform(MRI_array_datasets)


In [ ]:
import torchvision.models as models

data = MRI_array_datasets
# 确保数据是浮点类型
data = data.float()

# data = data.repeat(1, 3, 1, 1)  # 重复通道维度以匹配3通道的输入
# data = torch.nn.functional.interpolate(data, size=(32, 32), mode='bilinear', align_corners=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = data.to(device)

resnet50 = models.resnet50(pretrained=True)
resnet50 = resnet50.to(device)

resnet50.eval()
children = list(resnet50.children())

del children[-1]

resnet50 = torch.nn.Sequential(*children)
with torch.no_grad():
    # 获取特征
    features = resnet50(data)

features = features.squeeze(2)  # 移除单维度的高度维度
print("Feature shape:", features.shape)

In [ ]:
pd.DataFrame(features.reshape(929,2048).cpu()).to_csv("DL.csv",index=False)

In [ ]:
#肿瘤、瘤周、DL特征读取

external_valid_tumor_feature = pd.read_csv("./valid_pyra_slice_resample[555]_normal.csv")
external_valid_border_feature = pd.read_csv("./valid_border1_pyra_slice_resample[555]_normal.csv")
external_valid_DL_feature = pd.read_csv("./external_valid_resample[555]_feature_resnet50_NoPretrain_3.csv")

# label = external_valid_DL_feature.values[:,1]

# external_valid_tumor_feature = external_valid_tumor_feature.values[:,1:]
# external_valid_border_feature = external_valid_border_feature.values[:,1:]
# external_valid_DL_feature = external_valid_DL_feature.values[:,2:]

#初次筛选三类组学的下标读取
tumor_select = pd.read_csv("./tumor_select.csv")
border_select = pd.read_csv("./border_select.csv")
DL_select = pd.read_csv("./DL_select.csv")

In [ ]:
external_valid_tumor_select_feature = []
for i in range(len(tumor_select.columns[1:])):
    k = int(tumor_select.columns[1:][i])
    external_valid_tumor_select_feature.append(external_valid_tumor_feature.values[:,k+1])

external_valid_tumor_select_feature = np.array(external_valid_tumor_select_feature).T

external_valid_tumor_border_select_feature = []
for i in range(len(border_select.columns[1:])):
    k = int(border_select.columns[1:][i])
    external_valid_tumor_border_select_feature.append(external_valid_border_feature.values[:,k+1])

external_valid_tumor_border_select_feature = np.array(external_valid_tumor_border_select_feature).T

external_DL_select_feature = []
for i in range(len(DL_select.columns[2:])):
    k = int(DL_select.columns[2:][i])
    external_DL_select_feature.append(external_valid_DL_feature.values[:,k+2])

external_DL_select_feature = np.array(external_DL_select_feature).T



In [ ]:
standard_scaler = StandardScaler()
min_max_scaler = MinMaxScaler(feature_range=(-1,1))
external_valid_tumor_select_feature = standard_scaler.fit_transform(external_valid_tumor_select_feature)
external_valid_tumor_select_feature = min_max_scaler.fit_transform(external_valid_tumor_select_feature)


In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier,ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans

def Machine_learning(feature_inner, label_inner, feature_outer, label_outer,model):
    np.random.seed(42)

    best_auc = 0.0
    valid_max = []
    test_max = []


    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    test_auc_total = []
    valid_auc_total = []

    for train, test in skf.split(feature_inner, label_inner):
        data_train, data_test = feature_inner[train], feature_inner[test]
        label_train, label_test = label_inner[train], label_inner[test]

        # 在训练集上训练模型

        model.fit(data_train, label_train)
        test_prob = model.predict_proba(data_test)[:, 1]  # 得到正类别的概率
        test_auc = roc_auc_score(label_test, test_prob)

        valid_prob = model.predict_proba(feature_outer)[:, 1]
        valid_auc = roc_auc_score(label_outer, valid_prob)

        test_auc_total.append(test_auc)
        valid_auc_total.append(valid_auc)

        # if test_auc >= valid_auc:
        #     test_auc_total.append(test_auc)
        #     valid_auc_total.append(valid_auc)

    print(f"{model.__class__.__name__} Test_AUC: ")
    print(test_auc_total)
    print(f"{model.__class__.__name__} Valid_AUC: ")
    print(valid_auc_total)
    # k = 0.0
    # for i in range(len(test_auc_total)):
    #     if (test_auc_total[i]+valid_auc_total[i])/2 > k:
    #         k = (test_auc_total[i]+valid_auc_total[i])/2
    # return k
    test_max = np.array(test_auc_total)
    valid_max = np.array(valid_auc_total)

    if test_max.max() >= valid_max.max():
        return (valid_max.max()+test_max.max())/2 
    else :
        a = 0.0
        return a


In [12]:

def feature_select_auc(feature_inner, label_inner, feature_outer, label_outer,model):

    best_auc = 0.0
    index = []
    #use_feature为当前记录下的特征
    for j in range(5):
        if j == 0:
            for i in range(feature_inner.shape[1]):
                model_feature = feature_inner[:,i].reshape(-1,1)
                valid_feature = feature_outer[:,i].reshape(-1,1)
                auc = Machine_learning(model_feature, label_inner, valid_feature, label_outer, model)
                if auc > best_auc:
                    best_auc = auc
                    use_feature = model_feature
                    use_valid_feature = valid_feature
                    a = i

            index.append(a)
            index = np.array(index)

        elif j == 1:
            for i in range(feature_inner.shape[1]):
                if i not in index:
                    model_feature = feature_inner[:,i].reshape(-1,1)
                    valid_feature = feature_outer[:,i].reshape(-1,1)
                    if Machine_learning(np.concatenate((use_feature, model_feature),axis=1), label_inner, np.concatenate((use_valid_feature, valid_feature),axis=1), label_outer, model) >= best_auc:
                    # if logistic_classic(np.concatenate((use_feature, model_feature),axis=1), label) >= best_auc:
                        best_auc = Machine_learning(np.concatenate((use_feature, model_feature),axis=1), label_inner, np.concatenate((use_valid_feature, valid_feature),axis=1), label_outer, model)
                        use_feature = np.concatenate((use_feature, model_feature),axis=1)
                        use_valid_feature = np.concatenate((use_valid_feature, valid_feature),axis=1)
                        index = np.concatenate((index,[i]))
        elif j == 2:
            for i in range(feature_inner.shape[1]):   
                if i in index:
                    if use_feature.shape[1] == 1:
                        break
                    mask = np.all(feature_inner[:,i] == use_feature.T, axis=1)
                    if Machine_learning(np.delete(use_feature, np.where(mask)[0], axis=1), label_inner, np.delete(use_valid_feature, np.where(mask)[0], axis=1),label_outer, model) > best_auc:
                        best_auc = Machine_learning(np.delete(use_feature, np.where(mask)[0], axis=1), label_inner, np.delete(use_valid_feature, np.where(mask)[0], axis=1),label_outer, model)
                        use_feature = np.delete(use_feature, np.where(mask)[0], axis=1)
                        use_valid_feature = np.delete(use_valid_feature, np.where(mask)[0], axis=1)
                        index = np.delete(index, np.where(mask)[0],axis=0)
                else:
                    if Machine_learning(np.concatenate((use_feature, feature_inner[:,i].reshape(-1,1)),axis=1), label_inner, np.concatenate((use_valid_feature, feature_outer[:,i].reshape(-1,1)),axis=1),label_outer, model) >= best_auc:
                        best_auc = Machine_learning(np.concatenate((use_feature, feature_inner[:,i].reshape(-1,1)),axis=1), label_inner, np.concatenate((use_valid_feature, feature_outer[:,i].reshape(-1,1)),axis=1),label_outer, model)
                        use_feature = np.concatenate((use_feature, feature_inner[:,i].reshape(-1,1)),axis=1)
                        use_valid_feature = np.concatenate((use_valid_feature, feature_outer[:,i].reshape(-1,1)),axis=1)
                        index = np.concatenate((index,[i]))
        elif j == 3:
            for i in range(feature_inner.shape[1]):
                if i in index:
                    if use_feature.shape[1] == 1:
                        break
                    mask = np.all(feature_inner[:,i] == use_feature.T, axis=1)
                    if Machine_learning(np.delete(use_feature, np.where(mask)[0], axis=1), label_inner, np.delete(use_valid_feature, np.where(mask)[0], axis=1),label_outer, model) > best_auc:
                        Machine_learning(np.delete(use_feature, np.where(mask)[0], axis=1), label_inner, np.delete(use_valid_feature, np.where(mask)[0], axis=1),label_outer, model)
                        use_feature = np.delete(use_feature, np.where(mask)[0], axis=1)
                        use_valid_feature = np.delete(use_valid_feature, np.where(mask)[0], axis=1)
                        index = np.delete(index, np.where(mask)[0],axis=0)
        elif j == 4:
            for i in range(feature_inner.shape[1]):
                if i in index:
                    if use_feature.shape[1] == 1:
                        break
                    mask = np.all(feature_inner[:,i] == use_feature.T, axis=1)
                    if Machine_learning(np.delete(use_feature, np.where(mask)[0], axis=1), label_inner, np.delete(use_valid_feature, np.where(mask)[0], axis=1),label_outer, model) >= best_auc:
                        best_auc = Machine_learning(np.delete(use_feature, np.where(mask)[0], axis=1), label_inner, np.delete(use_valid_feature, np.where(mask)[0], axis=1),label_outer, model)
                        use_feature = np.delete(use_feature, np.where(mask)[0], axis=1)
                        use_valid_feature = np.delete(use_valid_feature, np.where(mask)[0], axis=1)
                        index = np.delete(index, np.where(mask)[0],axis=0)
    
    return best_auc, use_feature, use_valid_feature, index

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier,ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

def Machine_Learning_alpha(feature_inner, label_inner, feature_outer, label_outer, model):

    A = []

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    

    for train, test in skf.split(feature_inner, label_inner):
        data_train, data_test = feature_inner[train], feature_inner[test]
        label_train, label_test = label_inner[train], label_inner[test]

        # 在训练集上训练模型

        model.fit(data_train, label_train)
        A.append(bootstrap_ci(model, data_test, label_test))
        A.append(bootstrap_ci(model, feature_outer, label_outer))

    
    return A
    


In [10]:
from sklearn.metrics import roc_curve, precision_score, recall_score, accuracy_score, roc_auc_score, f1_score, average_precision_score

def bootstrap_ci(model, X_test, y_test, num_bootstrap=1000, alpha=0.95):
    metrics = {
        "AUC": [],
        "F1": [],
        "Precision": [],
        "Recall": [],
        "Accuracy": [],
        "Average Precision (AP)": [],
    }
    n_size = len(y_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    # y_pred = model.predict(X_test)
    threshold = calculate_threshold(y_test, y_pred_proba, expected_metric="Youden", expected_value=0.9)
    
    y_pred = (y_pred_proba >= threshold).astype(int)
    for _ in range(num_bootstrap):
        indices = np.random.choice(range(n_size), size=n_size, replace=True)
        if len(np.unique(y_test[indices])) < 2:
            continue
        cm = confusion_matrix(y_test[indices], y_pred[indices])

        
        auc = roc_auc_score(y_test[indices], y_pred_proba[indices])
        f1 = f1_score(y_test[indices], y_pred[indices])
        precision = precision_score(y_test[indices], y_pred[indices])
        recall = recall_score(y_test[indices], y_pred[indices])
        accuracy = accuracy_score(y_test[indices], y_pred[indices])
        ap = average_precision_score(y_test[indices], y_pred_proba[indices])
        
        metrics["AUC"].append(auc)
        metrics["F1"].append(f1)
        metrics["Precision"].append(precision)
        metrics["Recall"].append(recall)
        metrics["Accuracy"].append(accuracy)
        metrics["Average Precision (AP)"].append(ap)

    ci_results = {}
    for metric, scores in metrics.items():
        lower = np.percentile(scores, ((1.0 - alpha) / 2.0) * 100)
        upper = np.percentile(scores, (alpha + ((1.0 - alpha) / 2.0)) * 100)
        ci_results[metric] = (np.mean(scores), (lower, upper))
    
    return ci_results

In [9]:
def calculate_threshold(y_true, y_score, expected_metric="Youden", expected_value=0.5):
    fpr, tpr, thresholds = roc_curve(y_true, y_score)

    if expected_metric == "Youden":
        j_scores = tpr - fpr
        j_ordered = sorted(zip(j_scores, thresholds), reverse=True)
        threshold = j_ordered[0][1]
    elif expected_metric.lower() == "sen" or expected_metric.lower() == "sensitivity":
        idx = np.where(tpr >= expected_value)[0]
        threshold = thresholds[idx[0]]
    else:
        raise ValueError("Not supported expected metric: {}".format(expected_metric))

    return threshold



In [8]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from sklearn.calibration import calibration_curve
import numpy as np

def plot_curves(models, X_data, X_label, Y_data, Y_label, X_ci_results, Y_ci_results, X_kflod_index, Y_kflod_index, judge):
    colors = plt.cm.get_cmap('tab10', len(models))

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    if judge == True:
        p = 'Test'
        ci_results = X_ci_results
        kflod_index = X_kflod_index
    else:
        p = 'Valid'
        ci_results = Y_ci_results
        kflod_index = Y_kflod_index

    plt.figure(figsize=(20, 40))

    # ROC曲线
    plt.subplot(5, 2, 1)
    for i, (name, model) in enumerate(models.items()):
        model_X_data = X_data[i]  
        model_Y_data = Y_data[i]
        j=0
        for train, test in skf.split(model_X_data, X_label):
            data_train, data_test = model_X_data[train], model_X_data[test]
            label_train, label_test = X_label[train], X_label[test]
            if j == kflod_index[i]:
                break
            j=j+1
        if judge == False:
            data_test, label_test = model_Y_data, Y_label
            
        #训练数据选择
        model.fit(data_train, label_train)
        y_pred_proba_train = model.predict_proba(data_test)[:, 1]

        fpr_train, tpr_train, _ = roc_curve(label_test, y_pred_proba_train)
        plt.plot(fpr_train, tpr_train, label=f'{name} {p} (AUC: {ci_results[name]["AUC"][0]:.3f} [{ci_results[name]["AUC"][1][0]:.3f}, {ci_results[name]["AUC"][1][1]:.3f}])', color=colors(i))
    plt.plot([0, 1], [0, 1], 'k--', lw=2)  # 'k'代表黑色，'--'代表虚线
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - {p}')
    plt.legend()

    # PR曲线
    plt.subplot(5, 2, 3)
    for i, (name, model) in enumerate(models.items()):
        model_X_data = X_data[i]  
        model_Y_data = Y_data[i]
        j=0
        for train, test in skf.split(model_X_data, X_label):
            data_train, data_test = model_X_data[train], model_X_data[test]
            label_train, label_test = X_label[train], X_label[test]
            if j == kflod_index[i]:
                break
            j=j+1
        if judge == False:
            data_test, label_test = model_Y_data, Y_label
            
        #训练数据选择
        model.fit(data_train, label_train)
        y_pred_proba_train = model.predict_proba(data_test)[:, 1]

        precision_train, recall_train, _ = precision_recall_curve(label_test, y_pred_proba_train)
        plt.plot(recall_train, precision_train, label=f'{name} {p} (AP: {ci_results[name]["Average Precision (AP)"][0]:.3f} [{ci_results[name]["Average Precision (AP)"][1][0]:.3f}, {ci_results[name]["Average Precision (AP)"][1][1]:.3f}])', color=colors(i))
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - {p}')
    plt.legend()

    #图表位置
    # plt.legend(loc='lower right')

    # 校准曲线
    plt.subplot(5, 2, 5)
    for i, (name, model) in enumerate(models.items()):
        model_X_data = X_data[i]  
        model_Y_data = Y_data[i]
        j=0
        for train, test in skf.split(model_X_data, X_label):
            data_train, data_test = model_X_data[train], model_X_data[test]
            label_train, label_test = X_label[train], X_label[test]
            if j == kflod_index[i]:
                break
            j=j+1
        if judge == False:
            data_test, label_test = model_Y_data, Y_label
            
        #训练数据选择
        model.fit(data_train, label_train)
        y_pred_proba_train = model.predict_proba(data_test)[:, 1]

        prob_true_train, prob_pred_train = calibration_curve(label_test, y_pred_proba_train, n_bins=10)
        plt.plot(prob_pred_train, prob_true_train, label=f'{name} {p}', color=colors(i))
    plt.xlabel('Mean Predicted Probability')
    plt.ylabel('Fraction of Positives')
    plt.title(f'Calibration Curve - {p}')
    plt.legend()

    # 决策曲线分析 (DCA)
    plt.subplot(5, 2, 7)
    thresholds = np.linspace(0.01, 0.99, 10)
    for i, (name, model) in enumerate(models.items()):
        model_X_data = X_data[i]  
        model_Y_data = Y_data[i]
        j=0
        for train, test in skf.split(model_X_data, X_label):
            data_train, data_test = model_X_data[train], model_X_data[test]
            label_train, label_test = X_label[train], X_label[test]
            if j == kflod_index[i]:
                break
            j=j+1
        if judge == False:
            data_test, label_test = model_Y_data, Y_label
            
        #训练数据选择
        model.fit(data_train, label_train)
        y_pred_proba_train = model.predict_proba(data_test)[:, 1]

        net_benefits_train = [net_benefit(y_pred_proba_train, label_test, threshold) for threshold in thresholds]
        plt.plot(thresholds, net_benefits_train, label=f'{name} {p}', color=colors(i))
    treat_all_train = [np.sum(label_test) / len(label_test) - threshold / (1 - threshold) for threshold in thresholds]
    treat_none_train = [0 for _ in thresholds]
    plt.plot(thresholds, treat_all_train, linestyle='--', color='gray', label=f'Treat All {p}')
    plt.plot(thresholds, treat_none_train, linestyle='--', color='black', label=f'Treat None {p}')
    plt.ylim([-1.0, 0.4])
    plt.xlabel('Threshold Probability')
    plt.ylabel('Net Benefit')
    plt.title(f'Decision Curve Analysis - {p}')
    plt.legend()

    # 模型预测概率分布
    plt.subplot(5, 2, 9)
    for i, (name, model) in enumerate(models.items()):
        model_X_data = X_data[i]  
        model_Y_data = Y_data[i]
        j=0
        for train, test in skf.split(model_X_data, X_label):
            data_train, data_test = model_X_data[train], model_X_data[test]
            label_train, label_test = X_label[train], X_label[test]
            if j == kflod_index[i]:
                break
            j=j+1
        if judge == False:
            data_test, label_test = model_Y_data, Y_label
            
        #训练数据选择
        model.fit(data_train, label_train)
        y_pred_proba_train = model.predict_proba(data_test)[:, 1]
        plt.hist(y_pred_proba_train, bins=20, alpha=0.5, label=f'{name} {p}', color=colors(i))
    plt.xlabel('Predicted Probability')
    plt.ylabel('Frequency')
    plt.title(f'Predicted Probability Distribution - {p}')
    plt.legend()

def net_benefit(probabilities, y_true, threshold):
        tp = np.sum((probabilities >= threshold) & (y_true == 1))
        fp = np.sum((probabilities >= threshold) & (y_true == 0))
        fn = np.sum((probabilities < threshold) & (y_true == 1))
        tn = np.sum((probabilities < threshold) & (y_true == 0))
        
        prevalence = (tp + fn) / (tp + fp + tn + fn)
        net_benefit = tp / (tp + fp + tn + fn) - (fp / (tp + fp + tn + fn)) * (threshold / (1 - threshold))
        return net_benefit

In [7]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from sklearn.calibration import calibration_curve
import numpy as np

def one_model_plot_curves(models, X_data, X_label, Y_data, Y_label, X_ci_results, Y_ci_results, X_kflod_index, Y_kflod_index, judge, select_files, valid_select_files):
    colors = plt.cm.get_cmap('tab10', len(X_data))

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    if judge == True:
        p = 'Test'
        ci_results = X_ci_results
        kflod_index = X_kflod_index
        name = select_files
    else:
        p = 'Valid'
        ci_results = Y_ci_results
        kflod_index = Y_kflod_index
        name = valid_select_files

    plt.figure(figsize=(20, 40))
    
    #混淆矩阵
    for i in range(len(X_data)):
        model_X_data = X_data[f'select_{i}'][0]
        model_Y_data = Y_data[f'valid_select_{i}'][0]
        j=0
        for train, test in skf.split(model_X_data, X_label):
            data_train, data_test = model_X_data[train], model_X_data[test]
            label_train, label_test = X_label[train], X_label[test]
            if j == kflod_index[i]:
                break
            j=j+1
        if judge == False:
            data_test, label_test = model_Y_data, Y_label
            
        #训练数据选择
        models.fit(data_train, label_train)
        y_pred_proba_train = models.predict_proba(data_test)[:, 1]
        y_pred = models.predict(data_test)
    
        # 计算混淆矩阵
        cm = confusion_matrix(label_test, y_pred)
        
        # 输出混淆矩阵
        print(f"Confusion Matrix for fold {i+1}:")
        print(cm)


    # ROC曲线
    plt.subplot(5, 2, 1)
    for i in range(len(X_data)):
        model_X_data = X_data[f'select_{i}'][0]
        model_Y_data = Y_data[f'valid_select_{i}'][0]
        j=0
        for train, test in skf.split(model_X_data, X_label):
            data_train, data_test = model_X_data[train], model_X_data[test]
            label_train, label_test = X_label[train], X_label[test]
            if j == kflod_index[i]:
                break
            j=j+1
        if judge == False:
            data_test, label_test = model_Y_data, Y_label
            
        #训练数据选择
        models.fit(data_train, label_train)
        y_pred_proba_train = models.predict_proba(data_test)[:, 1]

        fpr_train, tpr_train, _ = roc_curve(label_test, y_pred_proba_train)
        plt.plot(fpr_train, tpr_train, label=f'{name[i]} {p} (AUC: {ci_results[name[i]]["AUC"][0]:.3f} [{ci_results[name[i]]["AUC"][1][0]:.3f}, {ci_results[name[i]]["AUC"][1][1]:.3f}])', color=colors(i))
    plt.plot([0, 1], [0, 1], 'k--', lw=2)  # 'k'代表黑色，'--'代表虚线
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve - {p}')
    plt.legend()
    plt.savefig('roc_curve.svg', format='svg', bbox_inches='tight')
    # PR曲线
    plt.subplot(5, 2, 3)
    for i in range(len(X_data)):
        model_X_data = X_data[f'select_{i}'][0]
        model_Y_data = Y_data[f'valid_select_{i}'][0]
        j=0
        for train, test in skf.split(model_X_data, X_label):
            data_train, data_test = model_X_data[train], model_X_data[test]
            label_train, label_test = X_label[train], X_label[test]
            if j == kflod_index[i]:
                break
            j=j+1
        if judge == False:
            data_test, label_test = model_Y_data, Y_label
            
        #训练数据选择
        model.fit(data_train, label_train)
        y_pred_proba_train = model.predict_proba(data_test)[:, 1]

        precision_train, recall_train, _ = precision_recall_curve(label_test, y_pred_proba_train)
        plt.plot(recall_train, precision_train, label=f'{name[i]} {p} (AP: {ci_results[name[i]]["Average Precision (AP)"][0]:.3f} [{ci_results[name[i]]["Average Precision (AP)"][1][0]:.3f}, {ci_results[name[i]]["Average Precision (AP)"][1][1]:.3f}])', color=colors(i))
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - {p}')
    plt.legend()

    #图表位置
    # plt.legend(loc='lower right')

    # 校准曲线
    plt.subplot(5, 2, 5)
    for i in range(len(X_data)):
        model_X_data = X_data[f'select_{i}'][0]
        model_Y_data = Y_data[f'valid_select_{i}'][0]
        j=0
        for train, test in skf.split(model_X_data, X_label):
            data_train, data_test = model_X_data[train], model_X_data[test]
            label_train, label_test = X_label[train], X_label[test]
            if j == kflod_index[i]:
                break
            j=j+1
        if judge == False:
            data_test, label_test = model_Y_data, Y_label
            
        #训练数据选择
        model.fit(data_train, label_train)
        y_pred_proba_train = model.predict_proba(data_test)[:, 1]

        prob_true_train, prob_pred_train = calibration_curve(label_test, y_pred_proba_train, n_bins=10)
        plt.plot(prob_pred_train, prob_true_train, label=f'{name[i]} {p}', color=colors(i))
    plt.xlabel('Mean Predicted Probability')
    plt.ylabel('Fraction of Positives')
    plt.title(f'Calibration Curve - {p}')
    plt.legend()

    # 决策曲线分析 (DCA)
    plt.subplot(5, 2, 7)
    thresholds = np.linspace(0.01, 0.99, 10)
    for i in range(len(X_data)):
        model_X_data = X_data[f'select_{i}'][0]
        model_Y_data = Y_data[f'valid_select_{i}'][0]
        j=0
        for train, test in skf.split(model_X_data, X_label):
            data_train, data_test = model_X_data[train], model_X_data[test]
            label_train, label_test = X_label[train], X_label[test]
            if j == kflod_index[i]:
                break
            j=j+1
        if judge == False:
            data_test, label_test = model_Y_data, Y_label
            
        #训练数据选择
        model.fit(data_train, label_train)
        y_pred_proba_train = model.predict_proba(data_test)[:, 1]

        net_benefits_train = [net_benefit(y_pred_proba_train, label_test, threshold) for threshold in thresholds]
        plt.plot(thresholds, net_benefits_train, label=f'{name[i]} {p}', color=colors(i))
    treat_all_train = [np.sum(label_test) / len(label_test) - threshold / (1 - threshold) for threshold in thresholds]
    treat_none_train = [0 for _ in thresholds]
    plt.plot(thresholds, treat_all_train, linestyle='--', color='gray', label=f'Treat All {p}')
    plt.plot(thresholds, treat_none_train, linestyle='--', color='black', label=f'Treat None {p}')
    plt.ylim([-1.0, 0.4])
    plt.xlabel('Threshold Probability')
    plt.ylabel('Net Benefit')
    plt.title(f'Decision Curve Analysis - {p}')
    plt.legend()

    # 模型预测概率分布
    plt.subplot(5, 2, 9)
    for i in range(len(X_data)):
        model_X_data = X_data[f'select_{i}'][0]
        model_Y_data = Y_data[f'valid_select_{i}'][0]
        j=0
        for train, test in skf.split(model_X_data, X_label):
            data_train, data_test = model_X_data[train], model_X_data[test]
            label_train, label_test = X_label[train], X_label[test]
            if j == kflod_index[i]:
                break
            j=j+1
        if judge == False:
            data_test, label_test = model_Y_data, Y_label
            
        #训练数据选择
        model.fit(data_train, label_train)
        y_pred_proba_train = model.predict_proba(data_test)[:, 1]
        plt.hist(y_pred_proba_train, bins=20, alpha=0.5, label=f'{name[i]} {p}', color=colors(i))
    plt.xlabel('Predicted Probability')
    plt.ylabel('Frequency')
    plt.title(f'Predicted Probability Distribution - {p}')
    plt.legend()

def net_benefit(probabilities, y_true, threshold):
        tp = np.sum((probabilities >= threshold) & (y_true == 1))
        fp = np.sum((probabilities >= threshold) & (y_true == 0))
        fn = np.sum((probabilities < threshold) & (y_true == 1))
        tn = np.sum((probabilities < threshold) & (y_true == 0))
        
        prevalence = (tp + fn) / (tp + fp + tn + fn)
        net_benefit = tp / (tp + fp + tn + fn) - (fp / (tp + fp + tn + fn)) * (threshold / (1 - threshold))
        return net_benefit